In [1]:
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import json
import re


In [2]:
with open('game_links.json', 'r') as f:
    game_links = json.load(f)

print(f"Trovati {len(game_links)} link")

Trovati 1323 link


In [3]:
# Funzione per convertire il tempo (MM:SS) in secondi totali
def convert_time_to_seconds(time_str):
    try:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    except (ValueError, AttributeError):
        return 0  # In caso di errore, restituisci 0

In [4]:
# Configura Selenium
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

In [6]:
url_end = '/game-charts?period=All'
link = game_links[0]
teams = re.search(r'/game/([a-z\-]+)-\d+', link).group(1)

# Inizializza la lista per memorizzare i dati
shots_data = []

# Avvia il WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# URL della pagina
url = link + url_end
driver.get(url)

# Attendi qualche secondo per il caricamento della pagina
time.sleep(1)

# Ottieni l'HTML della pagina
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')

# Trova tutti gli elementi "shot" nella pagina
shot_elements = soup.find_all('g', class_='shot')

In [7]:
# Regex per estrarre le informazioni dai titoli
player_shot_pattern = r'^(MISS )?(.*?)(\d+\'|\d+\'\s\d+\")?\s(.*?)\s\((\d+)\sPTS\)(?:\s\((.*?)\s(\d+)\sAST\))?\s(Q\d+)\s-\s(\d+:\d+)$'
miss_shot_pattern = r'^MISS\s(.*?)(\d+\'|\d+\'\s\d+\")?\s(.*?)\s(Q\d+)\s-\s(\d+:\d+)$'

In [48]:
shot = shot_elements[98]

# Ottieni il titolo che contiene le informazioni sul tiro
title_text = shot.find('title').text.strip()
title_text

"Porter Jr. 2' Cutting Dunk Shot (5 PTS) (Gordon 1 AST)\nQ1 - 07:58"

In [49]:
path = shot.find('path')
if path and 'transform' in path.attrs:
    transform_text = path['transform']
    # Estrai le coordinate dal testo transform="translate(X, Y)"
    coords_match = re.search(r'translate\(([^,]+),\s*([^)]+)\)', transform_text)
    if coords_match:
        x_coord = float(coords_match.group(1))
        y_coord = float(coords_match.group(2))
    else:
        x_coord = None
        y_coord = None
else:
    x_coord = None
    y_coord = None


print(path,'\n',x_coord,'\n',y_coord)

<path d="M -5, 0 a 5,5 0 1,0 10,0 a 5,5 0 1,0 -10,0" fill="rgba(255,255,255, 0.1" stroke="rgba(10, 158, 117, 0.8)" stroke-width="3" transform="translate(248, 65)"></path> 
 248.0 
 65.0


In [50]:
match = re.search(player_shot_pattern, title_text)
print(match)

<re.Match object; span=(0, 65), match="Porter Jr. 2' Cutting Dunk Shot (5 PTS) (Gordon 1>


In [53]:
print(match.group(1))

None


In [55]:
print(match.group(2))

Porter


In [56]:
print(match.group(3))

None


In [57]:
print(match.group(3))

None


In [ ]:
##################################
##################################
##################################
##################################

for shot in shot_elements:
    # Ottieni il titolo che contiene le informazioni sul tiro
    title_text = shot.find('title').text.strip()

    # Ottieni le coordinate del tiro dal tag path
    path = shot.find('path')
    if path and 'transform' in path.attrs:
        transform_text = path['transform']
        # Estrai le coordinate dal testo transform="translate(X, Y)"
        coords_match = re.search(r'translate\(([^,]+),\s*([^)]+)\)', transform_text)
        if coords_match:
            x_coord = float(coords_match.group(1))
            y_coord = float(coords_match.group(2))
        else:
            x_coord = None
            y_coord = None
    else:
        x_coord = None
        y_coord = None

    # Analizza il titolo per estrarre le informazioni
    # Prova prima con il pattern per i tiri fatti
    match = re.search(player_shot_pattern, title_text)
    if match:
        is_miss = bool(match.group(1))
        player = match.group(2).strip()
        distance = match.group(3).strip() if match.group(3) else ""
        shot_type = match.group(4).strip()
        points = match.group(5).strip() if not is_miss else "0"
        assist_player = match.group(6).strip() if match.group(6) else ""
        assist_count = match.group(7) if match.group(7) else ""
        quarter = match.group(8).strip()
        time_remaining = match.group(9).strip()
        

        shots_data.append({
            'Game': teams,
            'Giocatore': player,
            'Tipo di tiro': f"{distance} {shot_type}".strip(),
            'Risultato': 'Miss' if is_miss else 'Made',
            'Coordinate (x, y)': f"({x_coord}, {y_coord})",
            'Assist': f"{assist_player} ({assist_count})" if assist_player else "",
            'Tempo': time_remaining,
            'Quarto': quarter,
            'Punti': points,
            'Tempo_secondi': convert_time_to_seconds(time_remaining)  # Aggiungiamo una colonna per l'ordinamento
        })
    else:
        # Prova con il pattern per i tiri mancati
        miss_match = re.search(miss_shot_pattern, title_text)
        if miss_match:
            player = miss_match.group(1).strip()
            distance = miss_match.group(2).strip() if miss_match.group(2) else ""
            shot_type = miss_match.group(3).strip()
            quarter = miss_match.group(4).strip()
            time_remaining = miss_match.group(5).strip()

            shots_data.append({
                'Game': teams,
                'Giocatore': player,
                'Tipo di tiro': f"{distance} {shot_type}".strip(),
                'Risultato': 'Miss',
                'Coordinate (x, y)': f"({x_coord}, {y_coord})",
                'Assist': "",
                'Tempo': time_remaining,
                'Quarto': quarter,
                'Punti': "0",
                'Tempo_secondi': convert_time_to_seconds(time_remaining)  # Aggiungiamo una colonna per l'ordinamento
            })
        else:
            # Se non è possibile fare match, aggiungi comunque i dati grezzi
            shots_data.append({
                'Titolo Originale': title_text,
                'Coordinate (x, y)': f"({x_coord}, {y_coord})"
            })

# Chiudi il driver
driver.quit()

In [17]:
# Applica la funzione di conversione del tempo
for shot in shots_data:
    if 'Tempo' in shot:
        shot['Tempo_secondi'] = convert_time_to_seconds(shot['Tempo'])

# Crea un DataFrame con i dati raccolti
df = pd.DataFrame(shots_data)

# Ordina il DataFrame in base al tempo (dal tempo più alto al più basso, cioè dall'inizio alla fine del quarto)
df = df.sort_values(by=['Quarto', 'Tempo_secondi'], ascending=[True, False])
df = df.reset_index(drop=True)

# Rimuovi la colonna ausiliaria usata per l'ordinamento
if 'Tempo_secondi' in df.columns:
    df = df.drop('Tempo_secondi', axis=1)

# Salva il DataFrame in un file CSV
df.to_csv('shots_data.csv', index=False)

# Imposta un limite più ampio per la larghezza della visualizzazione
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
print(df)



print(f"Analisi completata. I dati sono stati salvati in 'shots_data.csv'")

           Game Giocatore                   Tipo di tiro Risultato  \
0    lal-vs-den      Paul             9' Fadeaway Jumper      Miss   
1    nop-vs-mem     Davis                        1' Dunk      Made   
2    lal-vs-den    Nurkic          4' Cutting Layup Shot      Made   
3    lal-vs-den     Curry    28' 3PT Step Back Jump Shot      Made   
4    nop-vs-mem     Jokic  7' Driving Floating Jump Shot      Made   
..          ...       ...                            ...       ...   
372  nop-vs-mem   Pickett           17' Pullup Jump Shot      Made   
373  lal-vs-den     Curry    24' 3PT Step Back Jump Shot      Made   
374  lal-vs-den    Nurkic          4' Cutting Layup Shot      Made   
375  lal-vs-den      Paul              24' 3PT Jump Shot      Miss   
376  lal-vs-den  Thompson              27' 3PT Jump Shot      Miss   

                            Coordinate (x, y)       Assist  Tempo Quarto Punti  
0                              (319.0, 110.0)               11:50     Q1     0